In [ ]:
!pip install --upgrade transformers

In [ ]:
import os
# Disable WandB by setting the environment variable
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import sys
if 'wandb' in sys.modules:
    del sys.modules['wandb']


In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')


In [ ]:
!pip install -q transformers datasets accelerate evaluate

import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate


In [ ]:
# Define dataset path
train_df= pd.read_excel("/content/drive/My Drive/Colab Notebooks/L3Cube-MahaHate/4-class/hate_train.xlsx")
test_df= pd.read_excel("/content/drive/My Drive/Colab Notebooks/L3Cube-MahaHate/4-class/hate_test.xlsx")
valid_df= pd.read_excel("/content/drive/My Drive/Colab Notebooks/L3Cube-MahaHate/4-class/hate_valid.xlsx")

In [ ]:
print("Train label distribution:\n", train_df['label'].value_counts())
print("Valid label distribution:\n", valid_df['label'].value_counts())
print("Test label distribution:\n", test_df['label'].value_counts())


In [ ]:
train_df

In [ ]:
train_df['label'] = train_df['label'].str.strip().str.upper()
valid_df['label'] = valid_df['label'].str.strip().str.upper()
test_df['label']  = test_df['label'].str.strip().str.upper()


In [ ]:
label_map = {'HATE': 0, 'OFFN': 1, 'PRFN': 2, 'NOT': 3}

train_df['label'] = train_df['label'].map(label_map)
valid_df['label'] = valid_df['label'].map(label_map)
test_df['label'] = test_df['label'].map(label_map)


In [ ]:
train_df

In [ ]:
train_df['label'] = train_df['label'].astype(int)
valid_df['label'] = valid_df['label'].astype(int)
test_df['label']  = test_df['label'].astype(int)

In [ ]:
print("Train labels:", train_df['label'].unique())
print("Valid labels:", valid_df['label'].unique())
print("Test labels:", test_df['label'].unique())


In [ ]:
print("Train label distribution:\n", train_df['label'].value_counts())
print("Valid label distribution:\n", valid_df['label'].value_counts())
print("Test label distribution:\n", test_df['label'].value_counts())


In [ ]:
from transformers import AutoTokenizer

# Load MahaHate-BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/mahahate-bert")

# Tokenize function
def tokenize_data(texts):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

# Tokenize each split
train_encodings = tokenize_data(train_df['text'])
valid_encodings = tokenize_data(valid_df['text'])
test_encodings  = tokenize_data(test_df['text'])


In [ ]:
import torch

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = HateSpeechDataset(train_encodings, train_df['label'])
valid_dataset = HateSpeechDataset(valid_encodings, valid_df['label'])
test_dataset  = HateSpeechDataset(test_encodings, test_df['label'])


In [ ]:
from transformers import BertForSequenceClassification

# Load the model while ignoring size mismatch in the classification head
model = BertForSequenceClassification.from_pretrained(
    "l3cube-pune/mahahate-bert",
    num_labels=4,
    ignore_mismatched_sizes=True
)


In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

# Compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    return {"accuracy": acc, "f1": f1}

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_steps=500,
    eval_steps=500,
    metric_for_best_model="accuracy",
    report_to="none",  # Disable reporting to wandb
)



# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
import os
# Disable WandB by setting the environment variable
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()


In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
model_save_path = "/content/drive/My Drive/MarathiHateModel"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)


In [ ]:
trainer.save_model("marathi-hate-speech-transformer")
tokenizer.save_pretrained("marathi-hate-speech-transformer")


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load from your saved directory
tokenizer = BertTokenizer.from_pretrained("marathi-hate-speech-transformer")
model = BertForSequenceClassification.from_pretrained("marathi-hate-speech-transformer")


In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)  # set device=-1 if no GPU


In [ ]:
samples = [
    "तू खूप चांगला आहेस",                 # Positive / NOT
    "मूर्ख आहेस तू",                     # Offensive
    "तुझ्यासारख्यांनी देश सोडावा",        # Hateful
    "बिनकामाचा मनुष्य"                  # Profane or Offensive
]


In [ ]:
results = classifier(samples)
for i, res in enumerate(results):
    print(f"Text: {samples[i]}")
    for label_score in res:
        print(f"  Label {label_score['label']}: {label_score['score']:.4f}")
    print("-" * 50)


In [ ]:
id2label = {
    "LABEL_0": "HATE",
    "LABEL_1": "OFFN",
    "LABEL_2": "PRFN",
    "LABEL_3": "NOT"
}


In [ ]:
for i, res in enumerate(results):
    print(f"Text: {samples[i]}")
    for label_score in res:
        label_name = id2label[label_score['label']]
        print(f"  Label {label_name}: {label_score['score']:.4f}")
    print("-" * 50)


In [ ]:
!pip install gradio --quiet
import gradio as gr


In [ ]:
from transformers import pipeline

# Load pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

# Label mapping
id2label = {
    "LABEL_0": "HATE",
    "LABEL_1": "OFFN",
    "LABEL_2": "PRFN",
    "LABEL_3": "NOT"
}

# Prediction function
def classify_text(text):
    results = classifier(text)[0]
    output = f"Text: {text}\n"
    for item in results:
        label = id2label[item['label']]
        score = round(item['score'], 4)
        output += f"Label {label}: {score:.4f}\n"
    return output


In [ ]:
demo = gr.Interface(
    fn=classify_text,
    inputs=gr.Textbox(label="Enter Marathi Text", placeholder="Type here...", lines=2),
    outputs=gr.Textbox(label="Prediction"),
    title="Marathi Hate Speech Detector",
    description="Click 'Check Toxicity' to see label-wise probabilities.",
    live=False,
    allow_flagging="never",
    theme="soft",
    examples=[
        ["मूर्ख आहेस तू"],
        ["तू खूप चांगला आहेस"],
        ["हे लोक देशद्रोही आहेत, त्यांना गोळी मारायला पाहिजे"],
        ["माझ्या आईने आज माझ्या आवडीचं जेवण केलं."],
        ["fu*k you, तुला काय कळतं?"],
        ["मला मराठी साहित्य वाचायला आवडतं."],

    ]
)

demo.launch()
